In [1]:
import torch,torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import pandas as pd
import wandb
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor,DefaultTrainer
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from tqdm import tqdm
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
PROJECT_NAME = 'Car-Object-Detection-V9-Learning-Detectron2-V2'

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
info = data.iloc[0]

In [4]:
img = cv2.imread(f'./data/{info["image"]}')
xmin = info['xmin']
ymin = info['ymin']
xmax = info['xmax']
ymax = info['ymax']

In [5]:
xmin,ymin,xmax,ymax

(281.2590449, 187.0350708, 327.7279305, 223.225547)

In [6]:
x = round(xmin)
y = round(ymin)
w = round(xmax - xmin)
h = round(ymax - ymin)

In [7]:
x,y,w,h

(281, 187, 46, 36)

In [8]:
crop = img[y:y+h,x:x+w]

In [9]:
plt.imshow(crop)

In [10]:
cv2.imwrite('./crop.png',crop)

True

In [11]:
plt.imshow(cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),2))

In [12]:
cv2.imwrite('./box.png',cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),2))

True

In [13]:
def load_data():
    if 'data.npy' in os.listdir('./'):
        return np.load('data.npy',allow_pickle=True)
    records = []
    for i in tqdm(range(len(data))):
        info = data.iloc[i]
        record = {}
        record['height'],record['width'] = cv2.imread(f'./data/{info["image"]}').shape[:2]
        record['file_name'] = f'./data/{info["image"]}'
        record['image_id'] = i
        record['annotations'] = [
            {
                'bbox':[info['xmin'],info['ymin'],info['xmax'],info['ymax']],
                'bbox_mode':BoxMode.XYXY_ABS,
                'category_id':0
            }
        ]
        records.append(record)
    np.save('./data.npy',records)
    return records

In [14]:
labels = ['Car']

In [15]:
DatasetCatalog.register('data',lambda : load_data())
MetadataCatalog.get('data').set(thing_classes=labels)
metadata = MetadataCatalog.get('data')

In [16]:
model = "COCO-Detection/faster_rcnn_R_50_C4_1x.yaml"

In [17]:
wandb.init(project=PROJECT_NAME,name='baseline')
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(model))
cfg.DATASETS.TRAIN = ('data')
cfg.DATASETS.TEST = ()
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.STEPS = []
cfg.SOLVER.MAX_ITER = 1250
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(labels)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator('data',output_dir='./output/')
val_loader = build_detection_test_loader(cfg,'data')
metrics = inference_on_dataset(predictor.model,val_loader,evaluator)
wandb.log(metrics)
wandb.finish()